In [1]:
import os

In [2]:
%pwd


'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy :str
    eval_steps : int
    save_steps : int
    gradient_accumation_steps : int



In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class configurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)  
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) ->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config= ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumation_steps = params.gradient_accumation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch

c:\Users\a12ej56k\AppData\Local\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-10 23:08:33,512 : INFO:config:PyTorch version 2.1.2 available.]


In [9]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading Data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir,num_train_epochs=self.config.num_train_epochs,warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size,
        #     per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay,logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy,eval_steps=self.config.evaluation_strategy,save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumation_steps
        # )


        trainer_args = TrainingArguments(
            output_dir=self.config.output_dir,num_train_epochs=1,warmup_steps=500,
            per_device_train_batch_size=1,per_device_eval_batch_size=1,
            weight_decay=0.01,logging_steps=10,evaluation_strategy='steps',
            eval_steps=500,save_steps=1e6,gradient_accumulation_steps=16
        )

        trainer = Trainer(model=model_pegasus,args=trainer_args,
                          tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        #save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum.model"))
        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [10]:
try:
    config = configurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-10 23:11:25,454 : INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-10 23:11:25,459 : INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-10 23:11:25,460 : INFO:common:created directory at: artifacts]


AttributeError: 'NoneType' object has no attribute 'root_dir'